In [2]:
import pyrender
import trimesh
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import mediapipe as mp
import Emotion
import warnings
warnings.filterwarnings("ignore")
import face_alignment_v1
fa = face_alignment_v1.FaceAlignment(face_alignment_v1.LandmarksType._3D, flip_input=False)
fa_2d = face_alignment_v1.FaceAlignment(face_alignment_v1.LandmarksType._2D, flip_input=False)
#import Age
opencv_home = cv2.__file__
folders = opencv_home.split(os.path.sep)[0:-1]
opencv_path = folders[0]
for folder in folders[1:]:
    opencv_path = opencv_path + "/" + folder
import tensorflow as tf
tf_version = tf.__version__
tf_major_version = int(tf_version.split(".", maxsplit=1)[0])
tf_minor_version = int(tf_version.split(".")[1])
if tf_major_version == 1:
    from keras.preprocessing import image as k_image
elif tf_major_version == 2:
    from tensorflow.keras.preprocessing import image as k_image

2023-04-07 15:52:50.590969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 15:52:54.607249: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/shared/CMPT/scratch/mka219/.conda/envs/image_segmentation/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.8/lib64:
2023-04-07 15:52:54.607888: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file

**MEDIAPIPE WITH PYRENDER**

In [3]:
# Load the glTF file
#model_trimesh = trimesh.load('/home/mka219/sfuhome/ar_project/data/ownfilter_without_v2.glb')
model_trimesh = trimesh.load('/home/mka219/sfuhome/ar_project/data/own_without_man.glb')
width = 1080
height = 720
model_meshes = []
vertices = []
faces = []
for geometry in model_trimesh.geometry.values():
    if isinstance(geometry, trimesh.Trimesh):
        model_meshes.append(geometry)
        vertices.append(geometry.vertices)
        faces.append(geometry.faces)
# Convert to pyrender Mesh object


model_mesh = pyrender.Mesh.from_trimesh(model_meshes)

scene = pyrender.Scene()
scene.add(model_mesh)

camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1)
camera_pose = np.eye(4)

camera_node = scene.add(camera, pose=camera_pose)

point_light = pyrender.PointLight(color=[1.0, 1.0, 1.0], intensity=10.0)
point_node = scene.add(point_light, pose=camera_pose)

light = pyrender.SpotLight(color=[1.0, 1.0, 1.0], intensity=10.0, innerConeAngle=np.pi/16, outerConeAngle=np.pi/6)

light_pose = np.eye(4)
light_pose[:3, 3] = [0.0, 0.0, 1.0]
light_node = scene.add(light, pose=camera_pose)

direction = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=1.0)
direction_node = scene.add(direction, pose=camera_pose)


renderer = pyrender.OffscreenRenderer(viewport_width=300, viewport_height=300)

In [4]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
face_mesh = mp_face_mesh.FaceMesh(min_tracking_confidence=0.1, min_detection_confidence=0.1)

sunglasses = cv2.imread("/home/mka219/sfuhome/ar_project/data/sunglasses.png", cv2.IMREAD_UNCHANGED)
neutral_filter = cv2.imread("/home/mka219/sfuhome/ar_project/comedy-glasses.png", cv2.IMREAD_UNCHANGED)
face_detector=cv2.CascadeClassifier(cv2.data.haarcascades +"/haarcascade_frontalface_default.xml")
emotion_detector = Emotion.loadModel()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2023-04-07 15:53:43.424271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-07 15:53:43.447404: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/shared/CMPT/scratch/mka219/.conda/envs/image_segmentation/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.8/lib64:
2023-04-07 15:53:43.447424: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to

In [33]:
cap = cv2.VideoCapture("/home/mka219/sfuhome/ar_project/input.mov")
# Define the camera intrinsics (focal length, principal point)
fx = width / (2 * np.tan(np.radians(63.5 / 2)))  # Focal length in x-axis (pixels)
fy = height / (2 * np.tan(np.radians(63.5 / 2)))
cx = width / 2  # Image center x-coordinate
cy = height / 2 

camera_matrix = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
], dtype=np.float32)

model_vertices = np.array([
            (1.672, -0.008797, -3.2535),     # mid eye
            (1.914, -0.004085, -4.2511),   # chin
            (1.6871, 0.924,  -2.3461), # Left eye corner
            (-1.6901 , -0.93014,  -2.3184), # Right eye corner
            (1.56 , 0.24,  -3.63),# Left mouth corner
            (-1.56 , -0.25,  3.63)  # Right mouth corner
        ], dtype=np.float32)

theta = np.radians(10)
Rotate_camera = np.eye(4)
Rotate_camera[:3, :3] = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])
theta = np.radians(90)
Rotate_camera = np.eye(4)
Rotate_camera[:3, :3] = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])

# Default camera distortion coefficients
dist_coeffs = np.array([0, 0, 0, 0], dtype=np.float32)

ind =0  
with open('output.txt', 'w') as f:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        color_img = frame
        gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)

        if result.multi_face_landmarks:
            # Get the first detected face
            face_landmarks = result.multi_face_landmarks[0]
        else:
            cv2.imshow('AR Filter', frame)
            continue
        
        faces, _, scores = face_detector.detectMultiScale3( gray_img, scaleFactor=1.1, minNeighbors=10, outputRejectLevels=True )
        if len(faces)==0:
            cv2.imshow('AR Filter', frame)
            continue
        #cv2.imwrite('coloredFrame.jpg', color)
        face = faces[0]
        gray_cropped_image = gray_img[face[1]:face[1]+face[3], face[0]:face[0]+face[2]]
        color_cropped_img = color_img[face[1]:face[1]+face[3], face[0]:face[0]+face[2]]
        factor_0 = 224 / color_cropped_img.shape[0]
        factor_1 = 224 / color_cropped_img.shape[1]
        factor = min(factor_0, factor_1)
        dsize = (int(color_cropped_img.shape[1] * factor), int(color_cropped_img.shape[0] * factor))
        color_cropped_img = cv2.resize(color_cropped_img, (224,224))
        img_pixels = k_image.img_to_array(color_cropped_img)  # what this line doing? must?
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255 

        img_gray = cv2.resize(gray_cropped_image, (48, 48))
        img_gray = np.expand_dims(img_gray, axis=0)
        # Emotion prediction
        emotion_predictions = emotion_detector.predict(img_gray, verbose=0)[0]
        emotion_predictions = Emotion.labels[np.argmax(emotion_predictions)]
        
        # Get the face landmarks

        if emotion_predictions=="happy":
            image_points = []
            object_points = []
            for i in [5, 175, 342, 156]: # [5, 175, 342, 156, 287, 57]
                image_points.append(np.array([face_landmarks.landmark[i].x * frame.shape[1],
                                                face_landmarks.landmark[i].y * frame.shape[0]]))
                object_points.append(np.array([face_landmarks.landmark[i].x ,face_landmarks.landmark[i].y , face_landmarks.landmark[i].z]))
            
            image_points = np.array(image_points, dtype=np.float32)
            object_points = np.array(object_points, dtype=np.float32)
            _, rvec, tvec = cv2.solvePnP(object_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_P3P)
            rotation_matrix_prev, _ = cv2.Rodrigues(rvec)
            #translation_vector = np.array([0.4, -3.5, 8])
            #translation_vector = np.array([tvec[0][0], tvec[1][0], tvec[2][0]])
            #print(translation_vector)
            image_points = []
            object_points = []

            for t in range(len(model_vertices)-2):
                object_points.append(np.matmul(rotation_matrix_prev, model_vertices[t])) 
            
            for i in [5, 175, 342, 156]:
                image_points.append(np.array([face_landmarks.landmark[i].x * frame.shape[1],
                                                face_landmarks.landmark[i].y * frame.shape[0]]))

            image_points = np.array(image_points, dtype=np.float32)
            object_points = np.array(object_points, dtype=np.float32)

            _, rvec, tvec = cv2.solvePnP(object_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_P3P)
            #rotation_matrix, _ = cv2.Rodrigues(rvec)
            translation_vector = np.array([0.4, -2.5, 7])
            
            new_camera_pose = np.eye(4)
            new_camera_pose[:3, :3] = rotation_matrix_prev
            new_camera_pose[:3, 3] = translation_vector
            #print(f"new_camera_pose: {new_camera_pose}", file=f)
            new_camera_pose = Rotate_camera @ new_camera_pose
            
            scene.set_pose(camera_node, pose=new_camera_pose)
            scene.set_pose(light_node, pose=new_camera_pose)
            scene.set_pose(point_node, pose=new_camera_pose)
            scene.set_pose(direction_node, pose=new_camera_pose)
            
        
            # Render 3D model onto image plane
            image, depth = renderer.render(scene, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.SHADOWS_DIRECTIONAL)
            image = np.uint8((image) * 255)
            # Overlay 3D model onto frame
            #alpha = 0
            mask = image[..., 3] > 0
            #frame = cv2.resize(frame, (1080, 720))
            max_depth = np.max(depth)
            mask = np.logical_and(depth > 0, depth < max_depth)
            mask = mask.astype(np.uint8) * 255

            frame[np.where(mask != 0)] = image[:,:,:3][np.where(mask != 0)]

        elif emotion_predictions=="surprise":
            # Neutral image
            sunglass_width = int(abs(face_landmarks.landmark[156].x * frame.shape[1]-face_landmarks.landmark[342].x * frame.shape[1])*1.3)
            sunglass_height = int(abs(face_landmarks.landmark[5].y * frame.shape[1]-face_landmarks.landmark[175].y * frame.shape[1])*0.6)
            sunglass_resized = cv2.resize(neutral_filter, (sunglass_width, sunglass_height), interpolation = cv2.INTER_CUBIC)
            t_height = sunglass_height//4
            x1, y1 = int(face_landmarks.landmark[156].x * frame.shape[1])-int((sunglass_width-abs(face_landmarks.landmark[156].x * frame.shape[1]-face_landmarks.landmark[342].x * frame.shape[1]))//2), t_height + int(face_landmarks.landmark[156].y * frame.shape[0]-sunglass_height//2)
            x2, y2 = int(face_landmarks.landmark[156].x * frame.shape[1])-int((sunglass_width-abs(face_landmarks.landmark[156].x * frame.shape[1]-face_landmarks.landmark[342].x * frame.shape[1]))//2)+sunglass_width, t_height+ int(face_landmarks.landmark[156].y * frame.shape[0]+(sunglass_height-sunglass_height//2))
            
            mask = sunglass_resized[:, :, 3] > 0
            
            # Replace the square in the original frame with the modified square
            frame[y1:y2, x1:x2][mask] = sunglass_resized[:, :, :3][mask]

        elif emotion_predictions=="neutral":
            
            sunglass_width = int(abs(face_landmarks.landmark[156].x * frame.shape[1]-face_landmarks.landmark[342].x * frame.shape[1])*1.3)
            sunglass_height = int((face_landmarks.landmark[5].x * frame.shape[1]-face_landmarks.landmark[156].x * frame.shape[1])/1.5)
            sunglass_resized = cv2.resize(sunglasses, (sunglass_width, sunglass_height), interpolation = cv2.INTER_CUBIC)

            x1, y1 = int(face_landmarks.landmark[156].x * frame.shape[1])-int((sunglass_width-abs(face_landmarks.landmark[156].x * frame.shape[1]-face_landmarks.landmark[342].x * frame.shape[1]))//2), int(face_landmarks.landmark[156].y * frame.shape[0]-sunglass_height//2)
            x2, y2 = int(face_landmarks.landmark[156].x * frame.shape[1])-int((sunglass_width-abs(face_landmarks.landmark[156].x * frame.shape[1]-face_landmarks.landmark[342].x * frame.shape[1]))//2)+sunglass_width, int(face_landmarks.landmark[156].y * frame.shape[0]+(sunglass_height-sunglass_height//2))
            
            mask = sunglass_resized[:, :, 3] > 0
            
            # Replace the square in the original frame with the modified square
            frame[y1:y2, x1:x2][mask] = sunglass_resized[:, :, :3][mask]
        
        org = (20, 40)
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 2
        color = (0, 0, 255)
        thickness = 3
        lineType = cv2.LINE_AA
        cv2.putText(frame, emotion_predictions, org, font, fontScale, color, thickness, lineType)

        #frame = cv2.circle(frame, (200,200), radius=2, color=(0, 0, 255), thickness=2)
        cv2.imwrite(f"/home/mka219/sfuhome/ar_project/output/frame{ind}.jpg", frame)
        cv2.imshow("AR Filter", frame)
        if cv2.waitKey(1) == ord("q"):
            break
        ind = ind + 1

    cap.release()
    cv2.destroyAllWindows()

In [10]:
import os
import cv2
width = 1080
height = 720

frames_dir = "/Users/manojkumar/Downloads/ar_project/output"
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.avi', 0, 25, (width, height))

# Loop through all the frames
for i in range(len(os.listdir(frames_dir))):
    # Load the frame
    filename = frames_dir+f"/frame{i}.jpg"
    frame = cv2.imread(filename)

    # Check if the frame was loaded successfully
    if frame is None or i>247:
        continue
            
    # Write the frame to the video
    out.write(frame)
    
# Release the VideoWriter object
out.release()

In [2]:
!pip install opencv-python

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv-python-4.3.0.38.tar.gz (88.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  ERROR: Command errored out with exit status 1:
   command: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python /Users/manojkumar/Library/Python/2.7/lib/python/site-packages/pip/_vendor/pep517/_in_process.py get_requires_for_build_wheel /var/folders/kb/y8jk7b3d4x9fmt0v9316l8h40000gn/T/tmpsXW_sy
       cwd: /private/var/folders/kb/y8jk7b3d4x